In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon
from dbfread import DBF
from jenkspy import jenks_breaks

In [2]:
#DATOS PARA PASO DOS

#INSERTAR MUN NECESARIOS
entidad = ['14']
municipios = ['002','039', '044', '051','070','097', '098', '101', '120', '124']

#RUTAS ENTRADA
MZNA_2020_ruta = r"C:\Users\UTIC\Documents\Alejandro\Docs\DATA\Censos\RESAGEBURB_14CSV20.csv"
ITER_2020_ruta = r"C:\Users\UTIC\Documents\Alejandro\Docs\DATA\Censos\ITER_14CSV20.csv"
MZNA_2010_ruta = r"C:\Users\UTIC\Documents\Alejandro\Docs\DATA\Censos\RESAGEBURB_14DBF10.dbf"
ITER_2010_ruta = r"C:\Users\UTIC\Documents\Alejandro\Docs\DATA\Censos\ITER_14DBF10.dbf"

#RUTAS SALIDA
MZNA_2020_salida = r"C:\Users\UTIC\Documents\Alejandro\Docs\DATA\TRATADA\AMG\Censos\CENSO_2020_AMG.csv"
MZNA_2010_salida = r"C:\Users\UTIC\Documents\Alejandro\Docs\DATA\TRATADA\AMG\Censos\CENSO_2010_AMG.csv"


In [ ]:
#PASO DOS - Preparación de información censal

# Función para cargar datos de archivo DBF y convertirlos a DataFrame
def load_dbf_to_dataframe(file_path):
    table = DBF(file_path)
    return pd.DataFrame(iter(table))

# Cargar archivos en un DataFrame
MZNA_2020_df = pd.read_csv(MZNA_2020_ruta, dtype={'ENTIDAD': str, 'MUN': str, 'LOC': str, 'AGEB': str, 'MZA': str})
ITER_2020_df = pd.read_csv(ITER_2020_ruta, dtype={'ENTIDAD': str, 'MUN': str, 'LOC': str, 'LATITUD': str})

MZNA_2010_df = load_dbf_to_dataframe(MZNA_2010_ruta)
ITER_2010_df = load_dbf_to_dataframe(ITER_2010_ruta)

print('añadir 0')

MZNA_2020_df['MUN'] = MZNA_2020_df['MUN'].str.zfill(3)
MZNA_2020_df['LOC'] = MZNA_2020_df['LOC'].str.zfill(4)
MZNA_2020_df['AGEB'] = MZNA_2020_df['AGEB'].str.zfill(4)
MZNA_2020_df['MZA'] = MZNA_2020_df['MZA'].str.zfill(3)

print('homologar')

# 2.2 Homologación de ID urbana - CVEGEO
def homologate_urban_id(row):
    return f"{row['ENTIDAD']}{row['MUN']}{row['LOC']}{row['AGEB']}{row['MZA']}"

# 3.2 Homologación de ID rural - CVEGEO
def homologate_rural_id(row):
    return f"{row['ENTIDAD']}{row['MUN']}{row['LOC']}"

MZNA_2020_df['CVEGEO'] = MZNA_2020_df.apply(homologate_urban_id, axis=1)
MZNA_2010_df['CVEGEO'] = MZNA_2010_df.apply(homologate_urban_id, axis=1)

ITER_2020_df['CVEGEO'] = ITER_2020_df.apply(homologate_rural_id, axis=1)
ITER_2010_df['CVEGEO'] = ITER_2010_df.apply(homologate_rural_id, axis=1)

print('Limp')

#reemplazar * con 0
def replace(dataframe):
    return dataframe.replace('*', '0')

MZNA_2020_df_rep = replace(MZNA_2020_df)
MZNA_2010_df_rep = replace(MZNA_2010_df)
ITER_2020_df_rep = replace(ITER_2020_df)
ITER_2010_df_rep = replace(ITER_2010_df)

print('Filtrar')

# Función para filtrar información por municipios correspondientes
def filtrar_municipios(dataframe, municipios):
    return dataframe[dataframe['MUN'].isin(municipios)]

# aplicar filtrado por municipios
MZNA_2020_df_filter = filtrar_municipios(MZNA_2020_df_rep, municipios)
MZNA_2010_df_filter = filtrar_municipios(MZNA_2010_df_rep, municipios)

#Filtrar ITER
def filtrar_municipios_iter(dataframe, municipios, entidad):
    return dataframe[(dataframe['MUN'].isin(municipios)) & (dataframe['ENTIDAD'].isin(entidad))]

ITER_2020_df_filter = filtrar_municipios_iter(ITER_2020_df_rep, municipios, entidad)
ITER_2010_df_filter = filtrar_municipios_iter(ITER_2010_df_rep, municipios, entidad)

#borrar columnas innesesarias
MZNA_2020_df_filter = MZNA_2020_df_filter.drop(columns=['ENTIDAD', 'NOM_ENT', 'MUN', 'NOM_MUN', 'LOC', 'NOM_LOC', 'AGEB', 'MZA'])
MZNA_2010_df_filter = MZNA_2010_df_filter.drop(columns=['ENTIDAD', 'NOM_ENT', 'MUN', 'NOM_MUN', 'LOC', 'NOM_LOC', 'AGEB', 'MZA'])

ITER_2020_df_filter = ITER_2020_df_filter.drop(columns=['ENTIDAD', 'NOM_ENT', 'MUN', 'NOM_MUN', 'LOC', 'NOM_LOC', 'LONGITUD', 'LATITUD', 'ALTITUD'])
ITER_2010_df_filter = ITER_2010_df_filter.drop(columns=['ENTIDAD', 'NOM_ENT', 'MUN', 'NOM_MUN', 'LOC', 'NOM_LOC', 'LONGITUD', 'LATITUD', 'ALTITUD'])

print(1)

#hacer columnas a número
cols1 = MZNA_2020_df_filter.columns.drop('CVEGEO')
cols2 = MZNA_2010_df_filter.columns.drop('CVEGEO')
cols3 = ITER_2020_df_filter.columns.drop('CVEGEO')
cols4 = ITER_2010_df_filter.columns.drop('CVEGEO')

MZNA_2020_df_filter[cols1] = MZNA_2020_df_filter[cols1].apply(pd.to_numeric, errors='coerce')
MZNA_2010_df_filter[cols2] = MZNA_2010_df_filter[cols2].apply(pd.to_numeric, errors='coerce')
ITER_2020_df_filter[cols3] = ITER_2020_df_filter[cols3].apply(pd.to_numeric, errors='coerce')
ITER_2010_df_filter[cols4] = ITER_2010_df_filter[cols4].apply(pd.to_numeric, errors='coerce')

CENSO_2020 = pd.concat([MZNA_2020_df_filter, ITER_2020_df_filter], ignore_index=True)
CENSO_2010 = pd.concat([MZNA_2010_df_filter, ITER_2010_df_filter], ignore_index=True)

print('guardar')

# 2.3 Guardado de información en CSV
def save_to_csv(dataframe, filename):
    dataframe.to_csv(filename, index=False)
    
save_to_csv(CENSO_2020, MZNA_2020_salida)
save_to_csv(CENSO_2010, MZNA_2010_salida)

print('fin')

In [70]:
#DATOS PARA PASO TRES A SEIS

#rutas shp base
SHP_MZN_ruta = r"C:\Users\UTIC\Documents\Alejandro\Docs\DATA\Marco Geo 20\14_jalisco\conjunto_de_datos\14m.shp"
SHP_LOC_ruta = r"C:\Users\UTIC\Documents\Alejandro\Docs\DATA\Marco Geo 20\14_jalisco\conjunto_de_datos\14l.shp"

#ruta gpkg fin
MAPA_TIPO_ruta = r"C:\Users\UTIC\Documents\Alejandro\Docs\DATA\TRATADA\AMG\MZNA_Censo_20_10.gpkg"

#INSERTAR MUN NECESARIOS
entidad = ['14']
municipios = ['002','039', '044', '051','070','097', '098', '101', '120', '124']

#Valor del filtro
filtro = 0.125

In [6]:
#PASO TRES - Unión de información estadística y cartografía

#Carga de capas de manzanas y localidades a gdf
SHP_MZN_gdf = gpd.read_file(SHP_MZN_ruta)
SHP_LOC_gdf = gpd.read_file(SHP_LOC_ruta)

#Filtrar ITER
def filtrar_municipios_gpkg(dataframe, municipios, entidad):
    return dataframe[(dataframe['CVE_MUN'].isin(municipios)) & (dataframe['CVE_ENT'].isin(entidad))]

SHP_MZN_gdf_filter = filtrar_municipios_gpkg(SHP_MZN_gdf, municipios, entidad)
SHP_LOC_gdf_filter = filtrar_municipios_gpkg(SHP_LOC_gdf, municipios, entidad)

# CREAR GPKG BASE DESDE gdf DE MANZANAS
SHP_MZN_gdf_filter.to_file(MAPA_TIPO_ruta, layer='MZNA_CENSO_20', driver='GPKG')

# Leer gpkg base
MAPA_TIPO_gdf = gpd.read_file(MAPA_TIPO_ruta, layer='MZNA_CENSO_20')

# Selección de ITER rural
SHP_LOC_gdf_filter_RURAL = SHP_LOC_gdf_filter[SHP_LOC_gdf_filter['AMBITO'] == 'Rural']

# Concatenar ITER rural a la capa existente
gdf_concatenated = gpd.GeoDataFrame(pd.concat([MAPA_TIPO_gdf, SHP_LOC_gdf_filter_RURAL], ignore_index=True))

# Guardar la capa concatenada en el GeoPackage base
gdf_concatenated.to_file(MAPA_TIPO_ruta, layer='MZNA_CENSO_20', driver='GPKG', overwrite=True)

# Re-leer la capa del GeoPackage base
MAPA_TIPO_gdf = gpd.read_file(MAPA_TIPO_ruta, layer='MZNA_CENSO_20')

#############################################
# Hacer un merge que no me requiera sumar individualmente las columnas del censo

print(1)

MAPA_TIPO_gdf20 = MAPA_TIPO_gdf.merge(CENSO_2020, on='CVEGEO', how='left')
MAPA_TIPO_gdf10 = MAPA_TIPO_gdf.merge(CENSO_2010, on='CVEGEO', how='left')

print(2)

#limpiar  *
MAPA_TIPO_gdf20 = MAPA_TIPO_gdf20.replace('*', np.nan)
MAPA_TIPO_gdf10 = MAPA_TIPO_gdf10.replace('*', np.nan)

#reemplazar N/D
MAPA_TIPO_gdf20 = MAPA_TIPO_gdf20.replace('N/D', np.nan)
MAPA_TIPO_gdf10 = MAPA_TIPO_gdf10.replace('N/D', np.nan)

# Actualizar valores sin información de "null" a 0
MAPA_TIPO_gdf20 = MAPA_TIPO_gdf20.fillna(0)
MAPA_TIPO_gdf10 = MAPA_TIPO_gdf10.fillna(0)

print(3)

# Guardar la capa concatenada en el GeoPackage base
MAPA_TIPO_gdf20.to_file(MAPA_TIPO_ruta, layer='MZNA_CENSO_20', driver='GPKG', overwrite=True)
MAPA_TIPO_gdf10.to_file(MAPA_TIPO_ruta, layer='MZNA_CENSO_10', driver='GPKG', overwrite=True)

print('fin')